In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the data using read_csv function from pandas package

df=pd.read_csv(r"C:\Users\ADMIN\Desktop\practise files\NLP\train.csv")
df.head()

,id,comment_text,toxic
0,e617e2489abe9bca,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0
1,9250cf637294e09d,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0
2,ce1aa4592d5240ca,"Marya Dzmitruk was born in Minsk, Belarus in M...",0
3,48105766ff7f075b,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0
4,0543d4f82e5470b6,New Categories \r\n\r\nI honestly think that w...,0


In [3]:
df.shape

(5000, 3)

In [4]:
df.isnull().sum()

id              0
comment_text    0
toxic           0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            5000 non-null   object
 1   comment_text  5000 non-null   object
 2   toxic         5000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [6]:
# Cleanup

i_p=r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
U_rl=r'((\w+)://([\w\-\.]+)/(\w+).(\w+))'
stop_words = set(stopwords.words('english'))
def preprocessing(DataFrame):
    global t
    t=DataFrame.comment_text
    for i in range(DataFrame.shape[0]):
        t[i]=re.sub(i_p,"",t[i])
        t[i]=re.sub(U_rl,"",t[i])
        t[i]=re.sub("'","",t[i])
        t[i]=re.sub("''","",t[i])
        t[i]=re.sub(r'\d+', '', t[i])
        t[i]=re.sub(r'[^\w\s]','',t[i])
        t[i] = re.sub(r'_','',t[i])
        t[i]=t[i].lower()
        t[i]=word_tokenize(t[i])
        t[i]=[words for words in t[i] if words not in stop_words]
        t[i] =[p for p in t[i] if p not in string.punctuation]
        t[i]=[lemmatizer.lemmatize(words) for words in t[i]]
        
preprocessing(df)
t[0][:5]

['barnstar', 'defender', 'wiki', 'barnstar', 'like']

In [7]:
# Using a counter, find the top terms in the data

most_common_words=[]
def most_common_words_list(list_collection):
    for word_list in list_collection:
        for (word, count) in Counter(word_list).most_common():
            if count>1:
                most_common_words.append(word)
                  

In [8]:
most_common_words_list(t)
len(most_common_words)


19360

In [9]:
Counter(most_common_words).most_common(10)

[('article', 425),
 ('page', 376),
 ('wikipedia', 238),
 ('please', 211),
 ('would', 180),
 ('talk', 178),
 ('one', 162),
 ('source', 131),
 ('dont', 123),
 ('like', 110)]

In [10]:
t=[" ".join(lists) for lists in t]
t[0]


'barnstar defender wiki barnstar like edit kayastha page let form solidarity group malign article subject matter propose folloing name group united intellectual front kayastha ethinicty racist castist abuse uifkearca'

In [11]:
# ELiminating contextual stopwords like “Wikipedia”, “page”, “edit” and those related to the same

t=[re.sub(r"article\w*","",sentence) for sentence in t]
t=[re.sub(r"wikipedia\w*","",sentence) for sentence in t]
t=[re.sub(r"edit\w*","",sentence) for sentence in t]
t=[re.sub(r"page\w*","",sentence) for sentence in t]
t[0]  

'barnstar defender wiki barnstar like  kayastha  let form solidarity group malign  subject matter propose folloing name group united intellectual front kayastha ethinicty racist castist abuse uifkearca'

In [12]:
df.comment_text=t
df.drop(columns="id",inplace=True)
df

,comment_text,toxic
0,barnstar defender wiki barnstar like kayastha...,0
1,seems unbalanced whatever said mathsci said fa...,0
2,marya dzmitruk born minsk belarus march mother...,0
3,talkback dear celestia,0
4,new category honestly think need add category ...,0
...,...,...
4995,dildo read response correctly never said going...,0
4996,calm calm dont get big dick,1
4997,opinion dougweller using privilege poorly pers...,0
4998,style section expanded didnt remember placed tag,0


In [13]:
df.toxic.value_counts()

0    4563
1     437
Name: toxic, dtype: int64

In [14]:
# Separate into train and test sets

X=df.comment_text
y=df.toxic

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=42)

In [18]:
X_train

1840    psyou really something life looked around user...
2115                        cant cant create  best friend
4437    blocked right posted suggested use time put ni...
1146    ga review review transcluded talkitalian cruis...
2486    please stop continue vandalize  blocked   plea...
                              ...                        
4426    yes accusation sock puppetry sock puppetry att...
466     curiosity  personal attack include strawman at...
3092    know got mutha fuckin charlie bronson must thi...
3772                                    ok seriously hell
860     go fuck bastard yyou life go fuck bastard yank...
Name: comment_text, Length: 3500, dtype: object

In [16]:
# Use TF-IDF values for the terms as feature to get into a vector space model

from sklearn.feature_extraction.text import TfidfVectorizer
vect=TfidfVectorizer(max_features=4000)
X_train_vect=vect.fit_transform(X_train)
X_test_vect=vect.transform(X_test)
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [17]:
X_train_vect

<3500x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 73448 stored elements in Compressed Sparse Row format>

In [87]:
y_train.shape

(3500,)

In [117]:
X_train_vect=X_train_vect.toarray()
X_test_vect=X_test_vect.toarray()


In [118]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(12, activation='relu', input_shape=(4000,)),
    Dense(15, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [119]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics='accuracy')

In [120]:
model.fit(X_train_vect, y_train,
batch_size=50, epochs=40, validation_split=0.3)


Epoch 1/40
49/49 [==============================] - 0s 4ms/step - loss: 0.6234 - accuracy: 0.8927 - val_loss: 0.5358 - val_accuracy: 0.9038
Epoch 2/40
49/49 [==============================] - 0s 2ms/step - loss: 0.4223 - accuracy: 0.9171 - val_loss: 0.3508 - val_accuracy: 0.9038
Epoch 3/40
49/49 [==============================] - 0s 2ms/step - loss: 0.2744 - accuracy: 0.9171 - val_loss: 0.2829 - val_accuracy: 0.9038
Epoch 4/40
49/49 [==============================] - 0s 2ms/step - loss: 0.2229 - accuracy: 0.9171 - val_loss: 0.2667 - val_accuracy: 0.9038
Epoch 5/40
49/49 [==============================] - 0s 2ms/step - loss: 0.1910 - accuracy: 0.9212 - val_loss: 0.2491 - val_accuracy: 0.9095
Epoch 6/40
49/49 [==============================] - 0s 2ms/step - loss: 0.1498 - accuracy: 0.9388 - val_loss: 0.2308 - val_accuracy: 0.9200
Epoch 7/40
49/49 [==============================] - 0s 2ms/step - loss: 0.1073 - accuracy: 0.9633 - val_loss: 0.2234 - val_accuracy: 0.9238
Epoch 8/40
49/49 [==

In [121]:
print(model.evaluate(X_test_vect,y_test))
print(model.metrics_names)

47/47 [==============================] - 0s 975us/step - loss: 0.3368 - accuracy: 0.9320
[0.3368034064769745, 0.9319999814033508]
['loss', 'accuracy']


In [136]:
df.loc[df["toxic"]==1]

,comment_text,toxic
7,loser cant block forever admin ego hippie freak,1
8,block block call dumbhead,1
16,there fucking wiki insane person,1
21,fuck assfuck assfuck assfuck assfuck assfuck a...,1
23,christian becaue jesus assyrian woman tit assy...,1
...,...,...
4933,ohcome guy nodq saw taping theyve never ever e...,1
4953,agree sophys duckling whoever wrote america po...,1
4977,sunshine quit asshole stating fact actually ap...,1
4985,yes complain rabbi go shoot defenseless arab t...,1


In [138]:
y_pred=model.predict(vect.transform([df.loc[21][0]]).toarray())
for val in y_pred:
    if val<0.5:
        print(0)
    else:
        print(1) 

1


In [73]:
vect.transform([X_train[0]]).shape

(1, 4000)

In [10]:
# Model building: Support Vector Machine

from sklearn.svm import SVC
svc=SVC(kernel='linear')
svc.fit(X_train_vect,y_train)
y_pred=svc.predict(X_test_vect)
y_pred[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [11]:
# Model evaluation: Accuracy, recall, and f1_score

from sklearn import metrics
print("accuracy ",metrics.accuracy_score(y_test, y_pred))
print("precision ",metrics.precision_score(y_test, y_pred))
print("recall ",metrics.recall_score(y_test, y_pred))
print("f1_score1 ",metrics.f1_score(y_test, y_pred))

accuracy  0.948
precision  0.9365079365079365
recall  0.44360902255639095
f1_score1  0.6020408163265306


In [94]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1367
           1       0.94      0.44      0.60       133

    accuracy                           0.95      1500
   macro avg       0.94      0.72      0.79      1500
weighted avg       0.95      0.95      0.94      1500



In [12]:
# Adjust the appropriate parameter in the SVC module to reduce the class imbalnce

svc=SVC(kernel='linear',class_weight="balanced")
svc.fit(X_train_vect,y_train)
y_pred=svc.predict(X_test_vect)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1367
           1       0.64      0.65      0.64       133

    accuracy                           0.94      1500
   macro avg       0.80      0.81      0.80      1500
weighted avg       0.94      0.94      0.94      1500



In [96]:
print("f1_score1 ",metrics.f1_score(y_test, y_pred))

f1_score1  0.6444444444444445


In [13]:
# Hyperparameter tuning

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)

param_grid={'C': [i/100 for i in range(5,16,1)]}
clf=GridSearchCV(estimator=svc,param_grid=param_grid, scoring= "recall",cv=skf) 



In [14]:
clf.fit(X_train_vect,y_train)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(class_weight='balanced', kernel='linear'),
             param_grid={'C': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12,
                               0.13, 0.14, 0.15]},
             scoring='recall')

In [15]:
# Find the parameters with the best recall in cross validation

clf.best_params_

{'C': 0.14}

In [16]:
svc=SVC(kernel='linear',class_weight="balanced", C=0.14)
svc.fit(X_train_vect,y_train)
y_pred=svc.predict(X_test_vect)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      1367
           1       0.66      0.72      0.69       133

    accuracy                           0.94      1500
   macro avg       0.82      0.84      0.83      1500
weighted avg       0.94      0.94      0.94      1500



In [18]:
metrics.f1_score(y_test, y_pred)

0.6881720430107527

In [17]:
# Predict and evaluate using the best estimator

svc=SVC(kernel='linear',class_weight="balanced", C=0.145)
svc.fit(X_train_vect,y_train)
y_pred=svc.predict(X_test_vect)
metrics.f1_score(y_test, y_pred)

0.6931407942238268

In [18]:
# Identify the most prominent terms in the toxic comments

d={"Comments":X_test,"Pred_Toxicity":y_pred}
df1=pd.DataFrame(data=d)
df1.loc[df1.Pred_Toxicity==1].head(10)

,Comments,Pred_Toxicity
1600,think bully course bully hate called anonymous...,1
794,back yeah know youre cahoot linux nutcase like...,1
4767,seriously soon asim unbanened wll wreacfk ever...,1
4628,redneck shouldnt bad mouth nascar people think...,1
2633,fucking album cover fuck supposed know owns fu...,1
4724,stop im done people like keep talking bad thin...,1
4524,quit threatening beyotch,1
4209,assume horrible nigger yuck,1
742,nigger nigger nigger nigger nigger nigger nigg...,1
3899,adijapan delete ethymologie constantly,1


In [19]:
toxic_list=list(df1.Comments)
toxic_list[0]

'agree asked add told likely deleted please talk contribs email'

In [28]:
toxic_list=[word_tokenize(sentence) for sentence in toxic_list]
toxic_word_list=[word for sentence in toxic_list for word in sentence]
toxic_word_list[0]

'agree'

In [33]:
Counter(toxic_word_list).most_common(15)

[('talk', 327),
 ('please', 312),
 ('one', 277),
 ('nigger', 265),
 ('like', 259),
 ('would', 250),
 ('dont', 237),
 ('source', 219),
 ('see', 217),
 ('think', 205),
 ('must', 202),
 ('people', 201),
 ('also', 189),
 ('time', 189),
 ('im', 178)]